<a href="https://colab.research.google.com/github/tinetor/Resume-screening/blob/main/ATS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Functions

In [1]:

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
def clean_text(x):  
  corpus = []
  for i in range(0, len(x)):
    review = re.sub('[^a-zA-Z]', ' ', x[i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)
  return corpus
def clean_text_(x):  
  corpus = []
 
  review = re.sub('[^a-zA-Z]', ' ', x)
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)
  return corpus 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

def bow(corpus):
    cv = CountVectorizer()
    vectors = cv.fit_transform(corpus).toarray()
    job_des=vectors[0]
    cv_=vectors[1]
    return job_des,cv_
def tf_df(corpus):
    cv = TfidfVectorizer()    
    vectors = cv.fit_transform(corpus).toarray()
    job_des_tf=vectors[0]
    cv_tf=vectors[1]
    return job_des_tf,cv_tf
def bert_(corpus):
    vectors= model.encode(corpus,convert_to_numpy=True)
    job_des=vectors[0]
    cv_=vectors[1]
    return job_des,cv_
    


     |████████████████████████████████| 71kB 5.6MB/s 
     |████████████████████████████████| 1.5MB 18.8MB/s 
     |████████████████████████████████| 1.1MB 38.8MB/s 
     |████████████████████████████████| 2.9MB 52.0MB/s 
     |████████████████████████████████| 890kB 38.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.0-cp36-none-any.whl size=102655 sha256=ae4da3a9cafd6620f98add55a35d56d922c998a8ad0415f55216a1e87e349704
  Stored in directory: /root/.cache/pip/wheels/ff/76/65/50258d8b7930e909ea2f5bd006a23d520a16765af13ab45bb3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=7ed4e3a35611309f9282ba34322791170925915029a4ae6ad28832a6e33e7f40
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


100%|██████████| 245M/245M [00:04<00:00, 49.6MB/s]


In [3]:
import numpy as np
import sklearn.metrics
import scipy
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
def distance(X,Y,C):
  corpuss=[X,Y]
  if C=='bow':
    job_des,cv=bow(clean_text(corpuss))
  if C=='tf_df':
    job_des,cv=tf_df(clean_text(corpuss))
  if C=='bert':
    job_des,cv=bert_(clean_text(corpuss))
  simi=sklearn.metrics.pairwise.cosine_similarity(job_des.reshape(1,-1), cv.reshape(1,-1))
  #simi=scipy.spatial.distance.cdist(job_des.reshape(1,-1), cv.reshape(1,-1),'cosine')
  return np.round(simi,2)

In [36]:
X='analyze key financial data and advise senior management on strategic planning.'
Y='Analyzing key financial data and advising senior management on strategic planning.'
Z='Participated in senior management business development and strategy meetings by presenting financial analysis of the business environment.'
H='Extracting contextual business intelligence from key datasets to advise on wider corporate strategy.'
J='Analyzed key financial data to recommend and advise the senior management on the strategic planning.'
K='The experience of being a Financial Analyst Strategist enabled me to develop the ability to analyze key financial data and stimulate the strategic planning of the decision making process.'
U='Analysed operating and financial data; wrote reports based on analysis for management to make key strategic decisions.'
A='Strong financial analysis skills and capacity to turn data into key strategic insights to support managerial decisions.'
B='Provided senior management with modeled raw data. Advised on corporate strategy in reliance on these data sets. '

Bag of Words, counts the number of times a word appear, doesnt consider the order.

In [37]:
sentences=[Y,Z,H,J,K,U,A,B]

Bag of Words

In [38]:
import pandas as pd

In [39]:
bow_=[]
for i in sentences:
  score=distance(X,i,'bow')
  bow_.append(score)

Term-frequency times inverse document-frequency

In [40]:
tf_df_=[]
for i in sentences:
  score=distance(X,i,'tf_df')
  tf_df_.append(score)

In [41]:
tf_df_

[array([[1.]]),
 array([[0.16]]),
 array([[0.12]]),
 array([[0.91]]),
 array([[0.38]]),
 array([[0.31]]),
 array([[0.23]]),
 array([[0.29]])]

Bert

In [42]:
bert__=[]
for i in sentences:
  score=distance(X,i,'bert')
  bert__.append(score)

In [43]:
results=pd.DataFrame(sentences)
results['Bag of Words']=bow_
results['Bag of Words'] = results['Bag of Words'].str[0].str[0]

In [44]:
results['tf_df']=tf_df_
results['tf_df']=results['tf_df'].str[0].str[0]

In [45]:
results['Bert']=bert__
results['Bert']=results['Bert'].str[0].str[0]

In [46]:
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [47]:
results.columns=['analyze key financial data and advise senior management on strategic planning.', 'Bag of Words', 'tf_idf', 'Bert']

# Results

In [48]:
results

,analyze key financial data and advise senior management on strategic planning.,Bag of Words,tf_idf,Bert
0,Analyzing key financial data and advising senior management on strategic planning.,1.00,1.00,1.00
1,Participated in senior management business development and strategy meetings by presenting financial analysis of the business environment.,0.27,0.16,0.69
2,Extracting contextual business intelligence from key datasets to advise on wider corporate strategy.,0.21,0.12,0.57
3,Analyzed key financial data to recommend and advise the senior management on the strategic planning.,0.95,0.91,0.91
4,The experience of being a Financial Analyst Strategist enabled me to develop the ability to analyze key financial data and stimulate the strategic planning of the decision making process.,0.54,0.38,0.76
5,Analysed operating and financial data; wrote reports based on analysis for management to make key strategic decisions.,0.46,0.31,0.77
6,Strong financial analysis skills and capacity to turn data into key strategic insights to support managerial decisions.,0.37,0.23,0.66
7,Provided senior management with modeled raw data. Advised on corporate strategy in reliance on these data sets.,0.45,0.29,0.58


In [49]:
D=clean_text_(X)
cleaned=results

In [50]:
cleaned.columns=[D[0],'Bag of Words', 'tf_idf', 'Bert']
cleaned.iloc[:,0]=cleaned.iloc[:,0].apply(clean_text_).str[0]

In [51]:
cleaned

,analyz key financi data advis senior manag strateg plan,Bag of Words,tf_idf,Bert
0,analyz key financi data advis senior manag strateg plan,1.00,1.00,1.00
1,particip senior manag busi develop strategi meet present financi analysi busi environ,0.27,0.16,0.69
2,extract contextu busi intellig key dataset advis wider corpor strategi,0.21,0.12,0.57
3,analyz key financi data recommend advis senior manag strateg plan,0.95,0.91,0.91
4,experi financi analyst strategist enabl develop abil analyz key financi data stimul strateg plan decis make process,0.54,0.38,0.76
5,analys oper financi data wrote report base analysi manag make key strateg decis,0.46,0.31,0.77
6,strong financi analysi skill capac turn data key strateg insight support manageri decis,0.37,0.23,0.66
7,provid senior manag model raw data advis corpor strategi relianc data set,0.45,0.29,0.58
